In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
nustatymai.add_argument('--log-level=3')  # Slopina nereikalingus pranešimus
driver = webdriver.Chrome(options=nustatymai)

pagrindinis_url = "https://www.trustedshops.de/shops/karneval_kostume/"
driver.get(pagrindinis_url)

wait = WebDriverWait(driver, 10)

urls = []  # Šiame sąraše kaupsime visus URL
puslapis = 1

while True:
    print(f"[INFO] Scrapinamas {puslapis}-as puslapis...")

    # Surandame visus produktų blokus
    try:
        produktai = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ShopResultItemstyles__ResultItem-sc-3gooul-0.dXCtjT")))
    except:
        print("[ERROR] Produktų nerasta! Tikriname, ar puslapis tikrai kraunasi...")
        break

    print(f"[INFO] Rasti {len(produktai)} produktai šiame puslapyje.")

    # Pereiname per kiekvieną produktą ir išsaugome nuorodą
    for produktas in produktai:
        url = produktas.get_attribute("href")
        if url and url not in urls:  # Apsauga nuo dublikatų ir tuščių URL
            urls.append(url)

    # Saugome dabartinį URL, kad patikrintume, ar jis keičiasi
    senas_url = driver.current_url

    try:
        # Tikriname, ar yra kitas puslapis
        print("[INFO] Tikriname, ar yra kitas puslapis...")
        kitas_puslapis = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".Linkstyles__LinkAsButton-sc-1h68u9x-1.Paginationstyles__PaginationItem-sc-1uibxtv-1.jwPbFO.gXvbNn.hide-on-small-mobile")))

        if "disabled" in kitas_puslapis.get_attribute("class"):
            print("[INFO] Paskutinis puslapis pasiektas. Išeiname iš ciklo.")
            break

        # Spaudžiame su JavaScript
        driver.execute_script("arguments[0].click();", kitas_puslapis)
        print("[INFO] Mygtukas paspaustas!")

        # Laukiame, kol URL pasikeis
        wait.until(EC.url_changes(senas_url))  # Tikriname, ar URL tikrai pasikeitė

        # Patikriname, ar URL pasikeitė
        naujas_url = driver.current_url
        if senas_url == naujas_url:
            print("[ERROR] Paspaudus mygtuką puslapis nepasikeitė. Paskutinis puslapis pasiektas.")
            break

        # Padidiname puslapio skaičių
        puslapis += 1

    except Exception as e:
        print(f"[INFO] Kito puslapio mygtukas nerastas. Paskutinis puslapis pasiektas. Klaida: {e}")
        break

# Įrašome URL į failą
failo_pavadinimas = "Karneval_Kostume_entdecken.txt"
with open(failo_pavadinimas, "w", encoding="utf-8") as f:
    for url in urls:
        f.write(url + "\n")

print(f"[INFO] Iš viso surinkta {len(urls)} URL'ų. Duomenys išsaugoti į '{failo_pavadinimas}'.")

driver.quit()




#     puslapis += 1

# # **Spausdiname rezultatą**
# print(f"[SUCCESS] Iš viso surinkta produktų: {len(urls)}")

# # **Išsaugome į failą**
# with open("produktai_urls.txt", "w", encoding="utf-8") as failas:
#     for url in urls:
#         failas.write(url + "\n")

# print("[INFO] URL sąrašas išsaugotas į 'produktai_urls.txt'!")

# driver.quit()

def reiksmes_filtras(driver, selektorius) :
    try :
        return driver.find_element(By.CSS_SELECTOR, selektorius).text
    except :
        return ""

data = []

for url in urls :
        driver.get(url)

        pavadinimas = reiksmes_filtras(driver, "h1.name")  

pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
print(pavadinimas)

        kaina = reiksmes_filtras(driver, ".price span") + "," + reiksmes_filtras(driver, ".price div sup")
        salis = reiksmes_filtras(driver, "ul.list:first-child .item:nth-child(1) p")
        prekes_zenklas = reiksmes_filtras(driver, "ul.list:first-child .item:nth-child(2) p")
        gamintojas = reiksmes_filtras(driver, "ul.list:first-child .item:nth-child(3) p")
        svoris = reiksmes_filtras(driver, "ul.list:first-child .item:nth-child(4) p")
        print(pavadinimas, kaina, salis, prekes_zenklas, gamintojas, svoris)

data.append(";".join([
    pavadinimas,
    kaina,
    salis,
    prekes_zenklas,
    gamintojas,
    svoris
]))

Kodas kuris ištaukia iš kategorijos url visus

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# Selenium nustatymai
options = Options()
# NEnaudosime headless, kad būtų matoma naršyklė
# options.add_argument("--headless") 

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")  # Atidarys pilno ekrano režimu
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# Pradinė URL nuoroda
base_url = "https://www.trustedshops.de/shops/baumarkt/?page="
urls = []
max_pages = 392  # Sustos ties 217 puslapiu

for current_page in range(1, max_pages + 1):
    full_url = base_url + str(current_page)
    driver.get(full_url)
    print(f"\n🔹 [INFO] Scrapinamas {current_page}-as puslapis ({full_url})")

    try:
        wait = WebDriverWait(driver, 15)
        product_links = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, ".ShopResultItemstyles__ResultItem-sc-3gooul-0.dXCtjT")
        ))

        page_urls = [product.get_attribute("href") for product in product_links if product.get_attribute("href")]
        print(f"✅ [INFO] Rasti {len(page_urls)} URL'ai šiame puslapyje.")

        if page_urls:
            print(f"🔍 [DEBUG] Pirmas URL šiame puslapyje: {page_urls[0]}")

        before_add = len(urls)
        urls.extend([url for url in page_urls if url not in urls])
        after_add = len(urls)
        print(f"🔄 [INFO] Pridėta {after_add - before_add} naujų URL. Iš viso surinkta: {len(urls)}")

    except Exception as e:
        print(f"❌ [ERROR] Klaida puslapyje {current_page}: {e}")

    wait_time = random.randint(8, 12)  # Atsitiktinė pauzė tarp 8-12 sekundžių
    print(f"⏳ [INFO] Laukiama {wait_time} sek. prieš kitą puslapį...\n")
    time.sleep(wait_time)  # Daro pertrauką, kad svetainė neblokuotų

print(f"\n📄 [INFO] Surinkta {len(urls)} URL'ų, išsaugota į 'Baumarkt.txt'.")

# Išsaugome į failą
file_name = "Baumarkt.txt"
with open(file_name, "w", encoding="utf-8") as f:
    for url in urls:
        f.write(url + "\n")

driver.quit()




🔹 [INFO] Scrapinamas 1-as puslapis (https://www.trustedshops.de/shops/baumarkt/?page=1)
✅ [INFO] Rasti 20 URL'ai šiame puslapyje.
🔍 [DEBUG] Pirmas URL šiame puslapyje: https://www.trustedshops.de/bewertung/info_XDD4FBB413ECE807A35AEC86AAD8EAD14.html
🔄 [INFO] Pridėta 20 naujų URL. Iš viso surinkta: 20
⏳ [INFO] Laukiama 12 sek. prieš kitą puslapį...


🔹 [INFO] Scrapinamas 2-as puslapis (https://www.trustedshops.de/shops/baumarkt/?page=2)
✅ [INFO] Rasti 20 URL'ai šiame puslapyje.
🔍 [DEBUG] Pirmas URL šiame puslapyje: https://www.trustedshops.de/bewertung/info_X1976B1222A6567BB090488A4AA96B352.html
🔄 [INFO] Pridėta 20 naujų URL. Iš viso surinkta: 40
⏳ [INFO] Laukiama 8 sek. prieš kitą puslapį...


🔹 [INFO] Scrapinamas 3-as puslapis (https://www.trustedshops.de/shops/baumarkt/?page=3)
✅ [INFO] Rasti 20 URL'ai šiame puslapyje.
🔍 [DEBUG] Pirmas URL šiame puslapyje: https://www.trustedshops.de/bewertung/info_X11298088E587E76A7FFC6B96B0D6FD6E.html
🔄 [INFO] Pridėta 20 naujų URL. Iš viso surinkt

Veikiantis kodas paima pavadinimą

In [45]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep

# Sukuriame naršyklę su pasirinkimais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Naršyklė veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# URL, kurį norime atidaryti
url = "https://www.trustedshops.de/bewertung/info_X378FA6FDE903D2783D081A33BA4F164F.html"
driver.get(url)

# Laukiame, kad puslapis užsikrautų
sleep(3)

# Pabandykime rasti pavadinimą naudojant selektorių
try:
    pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    print(f"Pavadinimas: {pavadinimas}")

    # Išsaugome pavadinimą į CSV failą
    with open("pavadinimai.csv", mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([pavadinimas])  # Įrašome pavadinimą į CSV failą
except Exception as e:
    print(f"Įvyko klaida: {e}")

# Baigiame naršyklės seansą
driver.quit()


Pavadinimas: PAYBACK


Veikiantis kodas paima aprašymą

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Sukuriame naršyklę
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone (galima išjungti debug'ui)
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Nustatykite URL
url = "https://www.trustedshops.de/bewertung/info_X378FA6FDE903D2783D081A33BA4F164F.html"
driver.get(url)
time.sleep(3)  # Palaukiame, kol puslapis užsikraus

# Paimti tekstą iš aprašymo pagal nurodytą klasę
try:
    aprašymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    print(f"Aprašymas: {aprašymas}")
except Exception as e:
    print(f"Klaida: {e}")

driver.quit()


Aprašymas: Erleben Sie eine Welt voller Trend- und Markenprodukte. Ob etwas kleines Praktisches, oder das überraschend Außergewöhnliche – die Prämienwelt von PAYBACK hat für jeden das Passende im Angebot. Belohnen Sie sich für Ihre gesammelten Punkte im PAYBACK-Prämienshop – mit garantiert tollen Angeboten!


Kodas ištraukia kontaktų pilną aprašymą kompanijos

In [71]:
from bs4 import BeautifulSoup
import requests

# Pateikiame URL
url = "https://www.trustedshops.de/bewertung/info_X09E759C6A97426A4937F97CDD8B4F8A1.html"

# Užklausos į svetainę
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Telefono numeris
phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
phone_number = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

# Svetainės adresas - ieškome nuorodų su 'http' arba 'https'
website = soup.find('a', href=True)
website_url = website['href'] if website and (website['href'].startswith('http://') or website['href'].startswith('https://')) else 'Svetainės adresas nerastas'

# Rasti visus elementus su klase 'contactInfo_companyContactDetailLink__OzJ99'
links = soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True)

# Inicializuojame svetainės adresą
website_url = 'Svetainės adresas nerastas'

# Tikriname kiekvieną nuorodą
for link in links:
    href = link['href']
    
    # Jei href prasideda su 'http' arba '//', tai svetainė
    if href.startswith('http') or href.startswith('//'):
        website_url = href if href.startswith('http') else 'http:' + href
        break  # Radome svetainę, daugiau nebereikia ieškoti

# El. paštas
email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
email_address = email.string if email else 'El. paštas nerastas'

# Adresas
# Surandame visus div elementus su klase 'contactInfo_companyContactDetail__d2tsS'
address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')

# Filtruojame, kad nebūtų telefono numerio ar svetainės
address_lines = []
for block in address_blocks:
    # Tikriname, ar nėra <a> nuorodų (nes jos yra telefono ir svetainės adresai)
    if not block.find('a'):
        # Skaldome viduje esančius elementus, kad nebūtų prilipusių žodžių
        parts = [el.strip() for el in block.stripped_strings]
        address_lines.extend(parts)  # Pridedame kiekvieną dalį atskirai

# Sudedame adresą su tarpais tarp eilučių
address_full = ' '.join(address_lines).replace('\xa0', ' ')  # \xa0 pašalina kietuosius tarpus

# Atspausdinkite ištrauktus duomenis
print("Telefonas:", phone_number)
print("Svetainė:", website_url)
print("El. paštas:", email_address)
print("Adresas:", address_full)


Telefonas: +4980036869358
Svetainė: http://www.rossmann-fotowelt.de
El. paštas: fotowelt@rossmann.de
Adresas: ORWO Net GmbH Röntgenstraße 3 06766 Bitterfeld-Wolfen Deutschland


Kodas randa svetainės adresą

In [ ]:
import requests
from bs4 import BeautifulSoup

# Pateikiame URL
url = "https://www.trustedshops.de/bewertung/info_X9780E98D167B6BA34F80D50738715136.html"

# Užklausos į svetainę
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Rasti visus elementus su klase 'contactInfo_companyContactDetailLink__OzJ99'
links = soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True)

# Inicializuojame svetainės adresą
website_url = 'Svetainės adresas nerastas'

# Tikriname kiekvieną nuorodą
for link in links:
    href = link['href']
    
    # Jei href prasideda su 'http' arba '//', tai svetainė
    if href.startswith('http') or href.startswith('//'):
        website_url = href if href.startswith('http') else 'http:' + href
        break  # Radome svetainę, daugiau nebereikia ieškoti

# Atspausdiname tik svetainės adresą
print("Svetainė:", website_url)


Svetainė: http://www.enjoyyourcamera.com


Kodas ištraukia adresą

In [69]:
import requests
from bs4 import BeautifulSoup

# Pateikiame URL
url = "https://www.trustedshops.de/bewertung/info_XCC56B1E1AD0AF5961A229D1E45BA1F18.html"

# Užklausos į svetainę
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Surandame visus div elementus su klase 'contactInfo_companyContactDetail__d2tsS'
address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')

# Filtruojame, kad nebūtų telefono numerio ar svetainės
address_lines = []
for block in address_blocks:
    # Tikriname, ar nėra <a> nuorodų (nes jos yra telefono ir svetainės adresai)
    if not block.find('a'):
        # Skaldome viduje esančius elementus, kad nebūtų prilipusių žodžių
        parts = [el.strip() for el in block.stripped_strings]
        address_lines.extend(parts)  # Pridedame kiekvieną dalį atskirai

# Sudedame adresą su tarpais tarp eilučių
address_full = ' '.join(address_lines).replace('\xa0', ' ')  # \xa0 pašalina kietuosius tarpus

# Atspausdiname tik įmonės adresą
print("Adresas:", address_full)


Adresas: Berliner Zinnfiguren Werner Scholtz e.K. Knesebeckstr. 88 10623 Berlin Deutschland


Veikiantis kodas, kuris iš vieno url paima visą reikiamą informaciją. 

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests

# Sukuriame naršyklę su pasirinkimais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone (galima išjungti debug'ui)
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# URL, kurį norime atidaryti
url = "https://www.trustedshops.de/bewertung/info_XAB537DF39AABA6E7BF60C330A2C36FFD.html"
driver.get(url)
time.sleep(3)  # Laukiame, kol puslapis užsikraus

# Pavadinimas
try:
    pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
except Exception as e:
    pavadinimas = 'Pavadinimas nerastas'
    print(f"Klaida renkant pavadinimą: {e}")

# Aprašymas
try:
    aprašymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
except Exception as e:
    aprašymas = 'Aprašymas nerastas'
    print(f"Klaida renkant aprašymą: {e}")

driver.quit()

# Užklausos į svetainę
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Telefono numeris
phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
phone_number = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

# Svetainės adresas
website = soup.find('a', href=True)
website_url = website['href'] if website and (website['href'].startswith('http://') or website['href'].startswith('https://')) else 'Svetainės adresas nerastas'

# Rasti visus elementus su klase 'contactInfo_companyContactDetailLink__OzJ99'
links = soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True)

# Inicializuojame svetainės adresą
website_url = 'Svetainės adresas nerastas'

# Tikriname kiekvieną nuorodą
for link in links:
    href = link['href']
    if href.startswith('http') or href.startswith('//'):
        website_url = href if href.startswith('http') else 'http:' + href
        break  # Radome svetainę, daugiau nebereikia ieškoti

# El. paštas
email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
email_address = email.string if email else 'El. paštas nerastas'

# Adresas
address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
address_lines = []
for block in address_blocks:
    if not block.find('a'):  # Nefiltruojame telefono ar svetainės
        parts = [el.strip() for el in block.stripped_strings]
        address_lines.extend(parts)

address_full = ' '.join(address_lines).replace('\xa0', ' ')  # Pašalina kietuosius tarpus

# Išsaugome informaciją į CSV failą
with open("informacija.csv", mode="a", newline="", encoding="utf-8") as failas:
    writer = csv.writer(failas)
    writer.writerow([pavadinimas, aprašymas, phone_number, website_url, email_address, address_full])  # Įrašome visą informaciją

# Atspausdinkite ištrauktus duomenis
print(f"Pavadinimas: {pavadinimas}")
print(f"Aprašymas: {aprašymas}")
print(f"Telefonas: {phone_number}")
print(f"Svetainė: {website_url}")
print(f"El. paštas: {email_address}")
print(f"Adresas: {address_full}")


Testas 1 į sql

In [ ]:
import csv
import sqlite3
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# Sukuriame arba prisijungiame prie SQLite duomenų bazės
conn = sqlite3.connect("imones_duomenys.db")
cursor = conn.cursor()

# Sukuriame lentelę, jei ji dar neegzistuoja
cursor.execute("""
CREATE TABLE IF NOT EXISTS imones (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    url TEXT,
    pavadinimas TEXT,
    aprasymas TEXT,
    telefonas TEXT,
    svetaine TEXT,
    el_pastas TEXT,
    adresas TEXT
)
""")
conn.commit()

# Perskaitome URL sąrašą iš failo
with open("surinkti_urls.txt", "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per kiekvieną URL
for url in url_sarasas:
    print(f"Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(3)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    links = soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True)
    svetaine = next((link['href'] if link['href'].startswith(('http', '//')) else 'http:' + link['href'] for link in links), 'Svetainės adresas nerastas')

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į duomenų bazę
    cursor.execute("""
        INSERT INTO imones (url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas))
    conn.commit()

    print(f"✅ Duomenys išsaugoti: {pavadinimas}")

# Uždarome naršyklę ir duomenų bazės ryšį
driver.quit()
conn.close()

print("🔄 Apdorojimas baigtas!")


Testas 2 į csv

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# CSV failo pavadinimas
csv_failas = "imones_duomenys.csv"

# Sukuriame CSV failą su stulpelių antraštėmis (jei jis dar neegzistuoja)
with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
    writer = csv.writer(failas)
    writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open("surinkti_urls.txt", "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per kiekvieną URL
for url in url_sarasas:
    print(f"Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(3)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė (taisytas variantas)
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ Duomenys išsaugoti: {pavadinimas}")

# Uždarome naršyklę
driver.quit()

print("🔄 Apdorojimas baigtas! Duomenys išsaugoti į imones_duomenys.csv ✅")


Kodas su sleep 5s

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# CSV failo pavadinimas
csv_failas = "imones_duomenys.csv"

# Sukuriame CSV failą su stulpelių antraštėmis (jei jis dar neegzistuoja)
with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
    writer = csv.writer(failas)
    writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open("surinkti_urls.txt", "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per kiekvieną URL
for url in url_sarasas:
    print(f"Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(3)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė (taisytas variantas)
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ Duomenys išsaugoti: {pavadinimas}")

    # Palaukiame 5 sekundes prieš pereinant prie kito URL
    time.sleep(30)

# Uždarome naršyklę
driver.quit()

print("🔄 Apdorojimas baigtas! Duomenys išsaugoti į imones_duomenys.csv ✅")


In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# CSV failo pavadinimas
csv_failas = "imones_duomenys.csv"

# Perskaitome jau surinktus URL iš CSV failo
existing_urls = set()
try:
    with open(csv_failas, mode="r", encoding="utf-8") as failas:
        reader = csv.reader(failas)
        next(reader)  # Praleidžiame antraštes
        for row in reader:
            if row:
                existing_urls.add(row[0])
except FileNotFoundError:
    # Jei failo nėra, sukuriame jį su antraštėmis
    with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open("surinkti_urls.txt", "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Filtruojame tik tuos URL, kurie dar nėra apdoroti
neapdoroti_urls = [url for url in url_sarasas if url not in existing_urls]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per neapdorotus URL
for url in neapdoroti_urls:
    print(f"Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(3)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ Duomenys išsaugoti: {pavadinimas}")

    # Palaukiame 5 sekundes prieš pereinant prie kito URL
    time.sleep(30)

# Uždarome naršyklę
driver.quit()

print("🔄 Apdorojimas baigtas! Duomenys išsaugoti į imones_duomenys.csv ✅")

Kodas kameval kategorija

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# Failų pavadinimai
txt_failas = "Karneval_Kostume_entdecken.txt"
csv_failas = "Karneval_Kostume_entdecken.csv"

# Perskaitome jau surinktus URL iš CSV failo
existing_urls = set()
try:
    with open(csv_failas, mode="r", encoding="utf-8") as failas:
        reader = csv.reader(failas)
        next(reader)  # Praleidžiame antraštes
        for row in reader:
            if row:
                existing_urls.add(row[0])
except FileNotFoundError:
    # Jei failo nėra, sukuriame jį su antraštėmis
    with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open(txt_failas, "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Filtruojame tik tuos URL, kurie dar nėra apdoroti
neapdoroti_urls = [url for url in url_sarasas if url not in existing_urls]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per neapdorotus URL
for url in neapdoroti_urls:
    print(f"🔹 [INFO] Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(30)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException:
        print(f"❌ [ERROR] Nepavyko pasiekti URL: {url}")
        continue  # Praleidžiame šį URL

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ [INFO] Duomenys išsaugoti: {pavadinimas}")

    # Palaukiame 30 sekundžių prieš pereinant prie kito URL
    time.sleep(30)

# Uždarome naršyklę
driver.quit()

print("🔄 [INFO] Apdorojimas baigtas! Duomenys išsaugoti į Karneval_Kostume_entdecken.csv ✅")


Kodas auto_motorarad_zubehor


In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# Failų pavadinimai
txt_failas = "Auto_Motorrad_Zubehor.txt"
csv_failas = "Auto_Motorrad_Zubehor.csv"

# Perskaitome jau surinktus URL iš CSV failo
existing_urls = set()
try:
    with open(csv_failas, mode="r", encoding="utf-8") as failas:
        reader = csv.reader(failas)
        next(reader)  # Praleidžiame antraštes
        for row in reader:
            if row:
                existing_urls.add(row[0])
except FileNotFoundError:
    # Jei failo nėra, sukuriame jį su antraštėmis
    with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open(txt_failas, "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Filtruojame tik tuos URL, kurie dar nėra apdoroti
neapdoroti_urls = [url for url in url_sarasas if url not in existing_urls]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per neapdorotus URL
for url in neapdoroti_urls:
    print(f"🔹 [INFO] Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(30)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException:
        print(f"❌ [ERROR] Nepavyko pasiekti URL: {url}")
        continue  # Praleidžiame šį URL

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ [INFO] Duomenys išsaugoti: {pavadinimas}")

    # Palaukiame 30 sekundžių prieš pereinant prie kito URL
    time.sleep(30)

# Uždarome naršyklę
driver.quit()

print("🔄 [INFO] Apdorojimas baigtas! Duomenys išsaugoti į Auto_Motorrad_Zubehor.csv ✅")

Drogerieartikel & Kosmetik kategorija

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# Failų pavadinimai
txt_failas = "Drogerieartikel_Kosmetik_entdecken.txt"
csv_failas = "Drogerieartikel_Kosmetik_entdecken.csv"

# Perskaitome jau surinktus URL iš CSV failo
existing_urls = set()
try:
    with open(csv_failas, mode="r", encoding="utf-8") as failas:
        reader = csv.reader(failas)
        next(reader)  # Praleidžiame antraštes
        for row in reader:
            if row:
                existing_urls.add(row[0])
except FileNotFoundError:
    # Jei failo nėra, sukuriame jį su antraštėmis
    with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open(txt_failas, "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Filtruojame tik tuos URL, kurie dar nėra apdoroti
neapdoroti_urls = [url for url in url_sarasas if url not in existing_urls]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per neapdorotus URL
for url in neapdoroti_urls:
    print(f"🔹 [INFO] Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(30)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException:
        print(f"❌ [ERROR] Nepavyko pasiekti URL: {url}")
        continue  # Praleidžiame šį URL

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ [INFO] Duomenys išsaugoti: {pavadinimas}")

    # Palaukiame 30 sekundžių prieš pereinant prie kito URL
    time.sleep(30)

# Uždarome naršyklę
driver.quit()

print("🔄 [INFO] Apdorojimas baigtas! Duomenys išsaugoti į Drogerieartikel_Kosmetik_entdecken.csv ✅")


Kodas kuris paima duomenis iš Bekleidung.txt url

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

# Failų pavadinimai
txt_failas = "Bekleidung.txt"
csv_failas = "Bekleidung.csv"

# Perskaitome jau surinktus URL iš CSV failo
existing_urls = set()
try:
    with open(csv_failas, mode="r", encoding="utf-8") as failas:
        reader = csv.reader(failas)
        next(reader)  # Praleidžiame antraštes
        for row in reader:
            if row:
                existing_urls.add(row[0])
except FileNotFoundError:
    # Jei failo nėra, sukuriame jį su antraštėmis
    with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open(txt_failas, "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Filtruojame tik tuos URL, kurie dar nėra apdoroti
neapdoroti_urls = [url for url in url_sarasas if url not in existing_urls]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# Keliaujame per neapdorotus URL
for url in neapdoroti_urls:
    print(f"🔹 [INFO] Apdorojamas URL: {url}")
    driver.get(url)
    time.sleep(30)  # Palaukiame, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = driver.find_element(By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span").text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = driver.find_element(By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span").text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Užklausos į svetainę su BeautifulSoup
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException:
        print(f"❌ [ERROR] Nepavyko pasiekti URL: {url}")
        continue  # Praleidžiame šį URL

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: x and x.startswith('+'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ [INFO] Duomenys išsaugoti: {pavadinimas}")

    # Palaukiame 30 sekundžių prieš pereinant prie kito URL
    time.sleep(30)

# Uždarome naršyklę
driver.quit()

print("🔄 [INFO] Apdorojimas baigtas! Duomenys išsaugoti į Drogerieartikel_Kosmetik_entdecken.csv ✅")


Computer_Unterhaltungselektronik_Zubehor.txt

In [ ]:
import csv
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Failų pavadinimai
txt_failas = "Consulting.txt"
csv_failas = "Consulting.csv"

# Perskaitome jau surinktus URL iš CSV failo
existing_urls = set()
try:
    with open(csv_failas, mode="r", encoding="utf-8") as failas:
        reader = csv.reader(failas)
        first_line = next(reader, None)  # Praleidžiame antraštes, jei yra
        if first_line is not None:  # Tikriname, ar failas ne tuščias
            for row in reader:
                if row:
                    existing_urls.add(row[0])
except FileNotFoundError:
    # Jei failo nėra, sukuriame jį su antraštėmis
    with open(csv_failas, mode="w", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow(["URL", "Pavadinimas", "Aprašymas", "Telefonas", "Svetainė", "El. paštas", "Adresas"])

# Perskaitome URL sąrašą iš failo
with open(txt_failas, "r", encoding="utf-8") as failas:
    url_sarasas = [line.strip() for line in failas.readlines()]

# Filtruojame tik tuos URL, kurie dar nėra apdoroti
neapdoroti_urls = [url for url in url_sarasas if url not in existing_urls]

# Sukuriame naršyklę su nustatymais
nustatymai = Options()
nustatymai.add_argument('--headless')  # Veikia fone
nustatymai.add_argument('--disable-blink-features=AutomationControlled')  # Mažina botų aptikimą
driver = webdriver.Chrome(options=nustatymai)

# HTTP antraštės, kad atrodytume kaip tikras vartotojas
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}

# Keliaujame per neapdorotus URL
for i, url in enumerate(neapdoroti_urls):
    print(f"🔹 [INFO] Apdorojamas URL: {url}")
    
    # Pirmiausia tikriname, ar URL pasiekiamas
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException:
        print(f"❌ [ERROR] Nepavyko pasiekti URL: {url}")
        continue

    # Naudojame Selenium tik jei reikia
    driver.get(url)
    time.sleep(30)  # Palaukiame 30 sekundžių, kol puslapis užsikraus

    # Pavadinimas
    try:
        pavadinimas = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-79691b37-3.eHuYIg span"))
        ).text
    except Exception:
        pavadinimas = "Pavadinimas nerastas"

    # Aprašymas
    try:
        aprasymas = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".companyDetails_companyDescription__rruNt span"))
        ).text
    except Exception:
        aprasymas = "Aprašymas nerastas"

    # Telefonas
    phone = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=lambda x: x and x.startswith('tel:'))
    telefonas = phone['href'].replace('tel:', '') if phone else 'Telefonas nerastas'

    # Svetainė
    svetaine = "Svetainės adresas nerastas"
    for link in soup.find_all('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True):
        href = link['href']
        if href.startswith("http") or href.startswith("//"):
            svetaine = href if href.startswith("http") else "http:" + href
            break

    # El. paštas
    email = soup.find('a', class_='contactInfo_companyContactDetailLink__OzJ99', href=True, string=lambda x: '@' in x)
    el_pastas = email.string if email else 'El. paštas nerastas'

    # Adresas
    address_blocks = soup.find_all('div', class_='contactInfo_companyContactDetail__d2tsS')
    adresas = ' '.join([el.strip() for block in address_blocks if not block.find('a') for el in block.stripped_strings]).replace('\xa0', ' ')

    # Išsaugome informaciją į CSV failą
    with open(csv_failas, mode="a", newline="", encoding="utf-8") as failas:
        writer = csv.writer(failas)
        writer.writerow([url, pavadinimas, aprasymas, telefonas, svetaine, el_pastas, adresas])

    print(f"✅ [INFO] Duomenys išsaugoti: {pavadinimas}")

    # Laukiame 30 sekundžių prieš kitą URL
    time.sleep(30)

    # Perkrauname naršyklę kas 50 URL, kad išvengtume atminties problemų
    if i % 50 == 0 and i != 0:
        driver.quit()
        driver = webdriver.Chrome(options=nustatymai)

# Uždarome naršyklę
driver.quit()

print("🔄 [INFO] Apdorojimas baigtas! Duomenys išsaugoti į Consulting.csv ✅")